# Project

In [35]:
# imports
import numpy as np
from scipy import sparse
import scipy.sparse.linalg
from matplotlib import pyplot as plt
import pandas as pd
import re

from sklearn.neighbors import kneighbors_graph
from sklearn.measure.pairwise import pairwise_kernels

## Data loading

In [2]:
DATA_FOLDER = 'data/'

In [41]:
# Load directed adjacency matrix
dir_adj = np.load(DATA_FOLDER + 'dir_adj_matrix.npy')
lc_index = np.load(DATA_FOLDER + 'largest_component_ix.npy')
dir_adj = dir_adj[lc_index, :][:,lc_index]
n_nodes = dir_adj[0]

# Load metadata
meta = pd.read_csv(DATA_FOLDER + 'categories.tsv', 
                   sep='\t', encoding='UTF-8', engine='python', comment='#', header=None)
meta.columns = ['Site','FullCategory']
meta = meta.iloc[lc_index]

# Extract top category
categories_re = np.copy(meta['FullCategory'])
for i in range(categories_re.size):
    tmp = categories_re[i]
    tmp = re.sub('subject.', '', tmp)
    tmp = re.sub('\..*', '', tmp)
    categories_re[i] = tmp

meta['TopCategory'] = categories_re

## Nearest Neighbor Graph Construction

### k Nearest Neighbors Construction
In the following, we infer the network based on the k-nearest-neighbors method. Thereby, every node is connected to the $k$ nearest neighbors in the space of our previously extracted keywords. $k$ is chosen to be the average degree of our original network, in order for the constructed network to be comparable to the original network.

In [42]:
def degree_distribution(adj):
    """Computes the degree distribution
    Params:
    adj: (directed) adjacency matrix

    Returns:
    a matrix with 2 columns, [in_degree, out_degree]
    """
    in_degree = np.matmul(adj.T,np.ones((adj.shape[1],1)))
    out_degree = np.matmul(adj,np.ones((adj.shape[0],1)))
    return in_degree, out_degree


In [43]:
in_degree, out_degree = degree_distribution(dir_adj)
avg_degree = in_degree.mean()

In [ ]:
knn_adj = kneighbors_graph(X, 
                           n_neighbors=avg_deg,    # I propose taking as many neighbors as is the avg deg
                           mode='connectivity',    # Return unweighted adjacency matrix
                           p=2,                    # Euclidian distance metric
                           n_jobs=-1               # Use all processors
                           )

### Kernel based graph construction

Here we construct a graph based on a kernel based distance measure: If the distance measure between two nodes is below a certain threshold, they are connected. 

In [ ]:
def kernel_graph_construction(X,  
                              metric='linear', cutoff=None, avg_deg=10, **kwds):
    pw_kernels = pairwise_kernels(X, metric=metric, **kwds)
    # TODO: Implement automatic cutoff detection
    out = [1 if i<=cutoff else 0 for i in pw_kernels]
    return out

## Pagerank applied to wikipedia articles


In [44]:
def page_rank(A,E,eps,maxit):
    '''
    A: Transition matrix of the graph (sparse)
    E: Random Surfer probabilities
    eps: stopping criterion ||R_new-R_old||1<eps
    maxit: maximum number of iterations   
    '''
    R = E
    maxit_stop = True
    for i in range(maxit):
        R_new = A.dot(E)
        d = np.linalg.norm(R_new,ord=1) - np.linalg.norm(R,ord=1)
        R_new = R_new + d*E
        if(np.linalg.norm(R_new-R)<eps):
            maxit_stop = false
    if(maxit_stop):
        print('Stopped due to maximum iteration condition')
    else:
        print('Stopped due to epsilon condition')
    